In [44]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
gsname = 'gs1102'

### load data

In [50]:
## run make_df-1102.py to make dfs
datadf = pd.read_csv('data/%s-datadf.csv'%gsname)
gsdf = pd.read_csv('data/%s-summdf.csv'%gsname)

In [47]:
datadf.columns
paramL = ['concentration', 'stickiness_wi', 'stickiness_bt',
       'sparsity', 'pvar', 'lrate', 'lratep', 'decay_rate']

### compute df metrics, form summary df

In [35]:
## subset of dataframe containing relevant params
Btest = gsdf.loc[:,'testacc-b'] > 0.95
Itest1 = gsdf.loc[:,'testacc-i'] > 0.5
Itest2 = gsdf.loc[:,'testacc-i'] < 0.62
Etest = gsdf.loc[:,'testacc-e'] > 0.8
Mtest1 = gsdf.loc[:,'testacc-m'] > 0.8
Mtest2 = gsdf.loc[:,'testacc-m'] < 0.9
Ltest = gsdf.loc[:,'testacc-l'] > 0.8
MItest = gsdf.loc[:,'testacc-m'] - gsdf.loc[:,'testacc-i'] > 0.1
LItest = gsdf.loc[:,'testacc-l'] - gsdf.loc[:,'testacc-i'] > 0.1

C = Btest&Itest1&Itest2&Mtest2
sgsdf = gsdf[C]
sgsdf.sort_values('testacc-m',ascending=False)

,Unnamed: 0,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,testacc-b,testacc-e,testacc-i,testacc-l,testacc-m
8859,0,59.968635,83.428028,96.439817,1.023505,0.0,1.0,1.0,1.0,0.978598,0.978590,0.500532,0.571651,0.711053
6262,0,41.565082,56.987626,64.292379,1.013430,0.0,1.0,1.0,1.0,0.978639,0.978630,0.503252,0.535953,0.675731
5065,0,33.520608,61.076372,71.447511,0.512834,0.0,1.0,1.0,1.0,0.980330,0.980338,0.505067,0.538793,0.661556
5593,0,36.947727,47.239177,68.608310,1.192392,0.0,1.0,1.0,1.0,0.978006,0.978006,0.503476,0.502590,0.651959
4854,0,32.014204,46.467018,83.784536,0.837269,0.0,1.0,1.0,1.0,0.979243,0.979229,0.506004,0.508285,0.642531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4412,0,29.326871,66.228869,54.808515,1.068048,0.0,1.0,1.0,1.0,0.978432,0.978447,0.500429,0.499472,0.496955
2179,0,14.614994,46.137124,27.880171,0.605218,0.0,1.0,1.0,1.0,0.980027,0.980011,0.501412,0.500233,0.496701
143,0,0.850139,1.317326,47.675404,0.098289,0.0,1.0,1.0,1.0,0.981698,0.981697,0.500953,0.502475,0.496360
2391,0,16.010283,21.463044,24.660197,0.820563,0.0,1.0,1.0,1.0,0.979308,0.979308,0.502425,0.504714,0.495717


### plt single condition

In [43]:
## select single param for plotting full acc trace
row = sgsdf.iloc[0]

query_str = " and ".join(
  ["%s == %f"%(k,v) for k,v in dict(row[:8].round(4)).items()]
)
print(query_str)
## df of single parameter condition
cond_df = datadf.round(4).query(query_str)

Unnamed: 0 == 0.000000 and concentration == 0.548400 and stickiness_wi == 23.093100 and stickiness_bt == 57.875300 and sparsity == 0.056200 and pvar == 0.000000 and lrate == 1.000000 and lratep == 1.000000


NotImplementedError: 'AnnAssign' nodes are not implemented

In [ ]:
f,axar = plt.subplots(1,1,figsize=(20,8))
ax=axar
for c,cdf in cond_df.groupby('cond'):
  ax.plot(np.arange(200),cdf.acc,label=c)
ax.set_ylim(0,1)
plt.legend()